In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist

In [3]:
# Load and preprocess the Fashion-MNIST dataset
(X_train, _), (_, _) = fashion_mnist.load_data()
X_train = (X_train - 127.5) / 127.5  # Normalize to [-1, 1] range
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension for compatibility

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Set constants
BATCH_SIZE = 32
NOISE_DIM = 100
EPOCHS = 50

In [5]:
# Define the Generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_shape=(NOISE_DIM,)),
        layers.ReLU(),
        layers.Dense(512),
        layers.ReLU(),
        layers.Dense(1024),
        layers.ReLU(),
        layers.Dense(784, activation='tanh'),
        layers.Reshape((28, 28, 1))
    ])
    return model

In [6]:
# Define the Discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(1024),
        layers.ReLU(),
        layers.Dropout(0.3),
        layers.Dense(512),
        layers.ReLU(),
        layers.Dropout(0.3),
        layers.Dense(256),
        layers.ReLU(),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [7]:
# Create generator and discriminator models
generator = build_generator()
discriminator = build_discriminator()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [9]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [10]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

In [11]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [12]:
# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [13]:
# Training loop with image display for each epoch
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

        # Display generated images at this stage
        print(f"Epoch {epoch+1}/{epochs}")
        display_images_comparison(generator)

In [14]:
def display_images_comparison(model, num_images=16):
    # Generate synthetic images
    noise = tf.random.normal([num_images, NOISE_DIM])
    generated_images = model(noise, training=False)

    # Display generated images
    print("Generated images:")
    fig, axs = plt.subplots(4, 4, figsize=(6, 6))
    for i, ax in enumerate(axs.flat):
        ax.imshow(generated_images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        ax.axis('off')
    plt.show()

    # Display real images for comparison
    print("Real images:")
    fig, axs = plt.subplots(4, 4, figsize=(6, 6))
    for i, ax in enumerate(axs.flat):
        ax.imshow(X_train[i].reshape(28, 28) * 127.5 + 127.5, cmap='gray')
        ax.axis('off')
    plt.show()

In [ ]:
# Create a TensorFlow dataset
BUFFER_SIZE = 60000
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Start training the GAN
train(train_dataset, EPOCHS)